In [ ]:
!pip install gymnasium

In [ ]:
from gymnasium import Env
from gymnasium.spaces import  Discrete
import numpy as np
import random
from scipy.stats import bernoulli
import yaml
from matplotlib import pyplot as plt
class RandomMaze(Env):
    def __init__(self,params):
        # action space left =0,up=1,right=2,down=3
        self.action_space = Discrete(4)
        # state spaces are 3,7-terminal, 5 wall, rest all non terminal
        self.observation_space=Discrete(12)
        # initial state is 8
        self.startState=params["startState"]
        self.state=self.startState
        # prob of 0.8 of going in the desired action direction
        self.goInDirection=params["goInDirection"]
        self.goOrthogonal=(1-self.goInDirection)/2
        # it incurs a livingCost for every non terminal state
        self.livingCost=params["livingCost"]

    def step(self,action):
        # stochasticity in the walk.
        # sample from uniform (0,1) distribution using random.random()
        random_num=np.random.random()
        if random_num<=self.goInDirection:
            pass
        else:
            bernouli_sample = np.random.binomial(1, .5)
            if bernouli_sample == 0:
                action= (action - 1)%4
            else:
                action= (action + 1)%4
        # left action 0
        if action == 0 :
            if self.state not in [0,4,8,6]:
                self.state-=1
            else:
                self.state=self.state
        # right action 2
        if action == 2 :
            if self.state!=4 and self.state!=11:
                self.state+=1
            else:
                self.state=self.state
        # up action =1
        if action == 1 :
            if self.state not in [0,1,2,9]:
                self.state-=4
            else:
                self.state=self.state
        # down action = 3
        if action == 3 :
            if self.state not in [1,8,9,10,11]:
                self.state+=4
            else:
                self.state=self.state

        if self.state==3:
            reward = 1
        elif self.state == 7:
            reward = -1
        else:
            reward=-self.livingCost
        # check wether we reached a terminal state or not
        if self.state==3 or self.state==7:
            done = True
        else:
            done = False
        # set placeholder for information
        info={}
        truncated = False
        return self.state,reward,done,truncated,info

    def render(self):
        pass

    def currState(self):
        return self.state

    def  reset(self, seed=None, options=None):
        # reset state
        super().reset(seed=seed)
        self.state=self.startState
        done=False
        info = {}
        return self.state,info


In [ ]:
from gymnasium import Env, register

register(id='RME-v0', entry_point=RandomMaze)

In [ ]:
import gymnasium as gym
import numpy as np
import math
import random
from matplotlib import pyplot as plt
random.seed(10)
np.random.seed(10)
params={}
params["gamma"]=0.99          #discount factor
params["goInDirection"]=0.8       #probability of successfully moving in the intended direction
params["livingCost"]=0.04         #living cost absolute value
params["startState"]=8            #start state
params["theta"]=pow(10,-10)       # threshold for convergence
params["S+"]=[0,1,2,3,4,6,7,8,9,10,11]        #all reachable states
params["S"]=[0,1,2,4,6,8,9,10,11]    #non terminal states
params["A"]=[0,1,2,3]                #action space
env=gym.make('RME-v0',params=params)
env.reset(seed=10)
done= False
score=0
time=0
while not done:
# env.render()
    action =random.choice([0,1,2,3])
    curr_state=env.currState()
    next_state, reward, done, truncated, info =env.step(action)
    #if done:
    #   env.reset(seed=10)
    score+=reward
    print('timeStamp:{}  CurrState:{}  Action:{}  NextState:{}  Reward:{}  Score:{:.2f}'.format(time,curr_state,action,next_state,reward,score))
    time+=1

In [ ]:
pi=np.zeros(12)
pi[0]=3
pi[1]=2
pi[2]=2
pi[3]=0   
pi[4]=1
pi[5]=0 
pi[6]=3
pi[7]=0 
pi[8]=1
pi[9]=2
pi[10]=3
pi[11]=3

In [ ]:

# creating the dynamics function for our random maze environment
l=-params["livingCost"]
# left =0,up=1,right=2,down=3
P={
    0:{
        0: [(0.8, 0, l, False), (0.1, 0, l, False), (0.1, 4, l, False)],
        1: [(0.8, 0, l, False), (0.1, 0, l, False), (0.1, 1, l, False)],
        2: [(0.8, 1, l, False), (0.1, 0, l, False), (0.1, 4, l, False)],
        3: [(0.8, 4, l, False), (0.1, 0, l, False), (0.1, 1, l, False)]
    },
    1:{
        0: [(0.8, 0, l, False), (0.1, 1, l, False), (0.1, 1, l, False)],
        1: [(0.8, 1, l, False), (0.1, 0, l, False), (0.1, 2, l, False)],
        2: [(0.8, 2, l, False), (0.1, 1, l, False), (0.1, 1, l, False)],
        3: [(0.8, 1, l, False), (0.1, 0, l, False), (0.1, 2, l, False)]
    },
    2:{
        0: [(0.8, 1, l, False), (0.1, 2, l, False), (0.1, 6, l, False)],
        1: [(0.8, 2, l, False), (0.1, 1, l, False), (0.1, 3, 1, True)],
        2: [(0.8, 3, 1, True), (0.1, 2, l, False), (0.1, 6, l, False)],
        3: [(0.8, 6, l, False), (0.1, 1, l, False), (0.1, 3, 1, True)]
    },
    3:{
        0: [(1.0, 0, 0.0, True)],
        1: [(1.0, 0, 0.0, True)],
        2: [(1.0, 0, 0.0, True)],
        3: [(1.0, 0, 0.0, True)]
    },
    4:{
        0: [(0.8, 4, l, False), (0.1, 0, l, False), (0.1, 8, l, False)],
        1: [(0.8, 0, l, False), (0.1, 4, l, False), (0.1, 4, l, False)],
        2: [(0.8, 4, l, False), (0.1, 0, l, False), (0.1, 8, l, False)],
        3: [(0.8, 8, l, False), (0.1, 4, l, False), (0.1, 4, l, False)]
    },
    6:{
        0: [(0.8, 6, l, False), (0.1, 2, l, False), (0.1, 10, l, False)],
        1: [(0.8, 2, l, False), (0.1, 6, l, False), (0.1, 7, -1, True)],
        2: [(0.8, 7, -1, True), (0.1, 2, l, False), (0.1, 10, l, False)],
        3: [(0.8, 10, l, False), (0.1, 6, l, False), (0.1, 7, -1, True)]
    },
    7:{
        0: [(1.0, 0, 0.0, True)],
        1: [(1.0, 0, 0.0, True)],
        2: [(1.0, 0, 0.0, True)],
        3: [(1.0, 0, 0.0, True)]
    },
    8:{
        0: [(0.8, 8, l, False), (0.1, 4, l, False), (0.1, 8, l, False)],
        1: [(0.8, 4, l, False), (0.1, 8, l, False), (0.1, 9, l, False)],
        2: [(0.8, 9, l, False), (0.1, 4, l, False), (0.1, 8, l, False)],
        3: [(0.8, 8, l, False), (0.1, 8, l, False), (0.1, 9, l, False)]
    },
    9:{
        0: [(0.8, 8, l, False), (0.1, 9, l, False), (0.1, 9, l, False)],
        1: [(0.8, 9, l, False), (0.1, 8, l, False), (0.1, 10, l, False)],
        2: [(0.8, 10, l, False), (0.1, 9, l, False), (0.1, 9, l, False)],
        3: [(0.8, 9, l, False), (0.1, 8, l, False), (0.1, 10, l, False)]
    },
    10:{
        0: [(0.8, 9, l, False), (0.1, 6, l, False), (0.1, 10, l, False)],
        1: [(0.8, 6, l, False), (0.1, 9, l, False), (0.1, 11, l, False)],
        2: [(0.8, 11, l, False), (0.1, 6, l, False), (0.1, 10, l, False)],
        3: [(0.8, 10, l, False), (0.1, 9, l, False), (0.1, 11, l, False)]
    },
    11:{
        0: [(0.8, 10, l, False), (0.1, 7, -1, True), (0.1, 11, l, False)],
        1: [(0.8, 7, -1, True), (0.1, 10, l, False), (0.1, 11, l, False)],
        2: [(0.8, 11, l, False), (0.1, 7, -1, True), (0.1, 11, l, False)],
        3: [(0.8, 11, l, False), (0.1, 10, l, False), (0.1, 11, l, False)]
    }
}


In [ ]:
'''
This function evaluates a given policy pi. It iteratively calculates the state-value function v for a policy until
the maximum change in value functions of any states is less than a small threshold theta. The input P is the transition
model, params contains the MDP parameters, and pi is the current policy.
The output v_new is the value function for the given policy, which estimates how good it is to be in each state following the policy.
'''
def PolicyEvaluation(P,params,pi):
    v_new=np.zeros(12)
    itr=0
    while 1:
        itr+=1
        delta = 0
        for s in params["S+"]:
            vs = 0
            for p,s_next,r,f in P[s][pi[s]]:  #as you are taking deterministic policy
                vs+=p*(r+params["gamma"]*v_new[s_next])
            # v_new[s]+=(pi[s]==a)*temp
            delta = max(delta, np.abs(v_new[s] - vs))
            v_new[s] = vs
        if delta < params["theta"]:
            break
    return v_new,itr


In [ ]:
'''
This function improves a given policy based on the value function v calculated from PolicyEvaluation.
It computes the action-value function Q for each action at each state and then updates the policy to choose
the action with the highest action-value at each state. The output pi is the improved policy.
'''

def PolicyImprovement(v,P,params):
    Q=np.zeros((12,len(params["A"])))
    pi=np.zeros(12)
    for s in params["S+"]:
        for a in params["A"]:
            for p,s_next,r,f in P[s][a]:
                Q[s][a]+=p*(r+params["gamma"]*v[s_next])
    for s in params["S+"]:
        pi[s]=np.argmax(Q[s])
    return pi


In [ ]:
'''
This function uses PolicyEvaluation and PolicyImprovement to find the optimal policy.
It alternates between evaluating a policy and improving it until the policy is stable (i.e., it no longer changes).
The output v is the value function for the optimal policy, pi is the optimal policy itself, and itr is the number of iterations it took to converge.
'''
def PolicyIteration(P,params,pi):
    itr=0
    while 1:
        pi_old=pi
        v,itrr=PolicyEvaluation(P,params,pi)
        itr+=itrr
        pi=PolicyImprovement(v,P,params)
        #can also have a value convergence check.  #TODO later
        if np.all(pi_old==pi):
            break
    return v,pi,itr
v,pi,itr=PolicyIteration(P,params,pi)
np.random.seed(3)
print("PolicyIteration")
print(v)
print(pi)
print(itr)


# left =0,up=1,right=2,down=3

In [ ]:
'''
This function initializes a value function v to zero for all states and iteratively computes
the action-value function Q for each state-action pair. The major computations involve updating Q
based on the expected rewards and the discounted value of subsequent states, as given by the Bellman equation.
The function iterates until the change in v is below theta, indicating convergence.
'''
def ValueIteration(P,params):
    v=np.zeros(12)
    itr=0
    while 1:
        delta = 0
        itr+=1
        Q=np.zeros((12,4))
        for s in params["S+"]:
            for a in params["A"]:
                for p,s_next,r,f in P[s][a]:
                    Q[s][a]+=p*(r+params["gamma"]*v[s_next])
        for s in params["S+"]:
            vs = v[s]
            v[s] = np.max(Q[s])
            delta = max(delta, np.abs(vs-v[s]))
        if delta < params["theta"]:
            break

    pi = PolicyImprovement(v, P, params)
    return v,pi,itr
np.random.seed(3)
v,pi,itr=ValueIteration(P,params)
print("value")
print(v)
print(pi)
print(itr)
